# Domain Adaptation Via Activation Shaping

**Teacher assistant** Iurada Leonardo

**Students**

- Bar Giorgio
- Distefano Giuseppe
- Incaviglia Salvatore

## Code

### Libraries

In [1]:
!pip install torch torchvision tqdm torchmetrics

import torch

import torch.backends.mps
from argparse import ArgumentParser

import os
import torchvision.transforms as T

from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CIFAR10

import numpy as np
import random
from PIL import Image

import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights

import torch.nn.functional as F
from torchmetrics import Accuracy
from tqdm import tqdm

import logging
import warnings

import argparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 6.7 MB/s eta 0:00:00


### Setup PACS dataset and environment

In [2]:
# Download PACS Dataset Images and Labels
!git clone https://github.com/MachineLearning2020/Homework3-PACS/
!git clone https://github.com/silvia1993/DANN_Template/

# Setup data
!rm -rf data || true
!rm -rf record || true
!mkdir data
!mkdir data/kfold
!cp -r Homework3-PACS/PACS/ data/kfold
!cp DANN_Template/txt_lists/*.txt data
!rm -rf Homework3-PACS/
!rm -rf DANN_Template/
!rm -rf sample_data || true

Cloning into 'Homework3-PACS'...
remote: Enumerating objects: 10032, done.
remote: Total 10032 (delta 0), reused 0 (delta 0), pack-reused 10032
Receiving objects: 100% (10032/10032), 174.13 MiB | 32.79 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (9993/9993), done.
Cloning into 'DANN_Template'...
remote: Enumerating objects: 23, done.
remote: Total 23 (delta 0), reused 0 (delta 0), pack-reused 23
Receiving objects: 100% (23/23), 33.86 KiB | 533.00 KiB/s, done.
Resolving deltas: 100% (5/5), done.


### Globals

#### globals

In [3]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

CONFIG = dotdict({})

if torch.cuda.is_available():
    CONFIG.device = 'cuda'
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    CONFIG.device = 'mps'
else:
    CONFIG.device = 'cpu'

CONFIG.dtype = torch.float32

#### parse args

In [4]:
def _clear_args(parsed_args):
    parsed_args.experiment_args = eval(parsed_args.experiment_args)
    parsed_args.dataset_args = eval(parsed_args.dataset_args)
    return parsed_args


def parse_arguments():
    parser = ArgumentParser()

    parser.add_argument('--seed', type=int, default=0, help='Seed used for deterministic behavior')
    parser.add_argument('--test_only', action='store_true', help='Whether to skip training')
    parser.add_argument('--cpu', action='store_true', help='Whether to force the usage of CPU')

    parser.add_argument('--experiment', type=str, default='baseline')
    parser.add_argument('--experiment_name', type=str, default='baseline')
    parser.add_argument('--experiment_args', type=str, default='{}')
    parser.add_argument('--dataset_args', type=str, default='{}')

    parser.add_argument('--batch_size', type=int, default=128)
    parser.add_argument('--epochs', type=int, default=30)
    parser.add_argument('--num_workers', type=int, default=5)
    parser.add_argument('--grad_accum_steps', type=int, default=1)

    return _clear_args(parser.parse_args())

#### Global variables

In [5]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLASSES = 7
BATCH_SIZE = 256
LR = 1e-3               # The initial Learning Rate
MOMENTUM = 0.9          # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5     # Regularization, you can keep this at the default
NUM_EPOCHS = 30         # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20          # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1             # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

### Dataset

#### utils

In [6]:
class BaseDataset(Dataset):
    def __init__(self, examples, transform):
        self.examples = examples
        self.T = transform


    def __len__(self):
        return len(self.examples)


    def __getitem__(self, index):
        x, y = self.examples[index]
        x = Image.open(x).convert('RGB')
        x = self.T(x).to(CONFIG.dtype)
        y = torch.tensor(y).long()
        return x, y

######################################################

class DomainAdaptationDataset(Dataset):
    def __init__(self, source_examples, target_examples, transform):
        self.source_examples = source_examples
        self.target_examples = target_examples
        self.T = transform


    def __len__(self):
        return len(self.source_examples)


    def __getitem__(self, index):
        src_x, src_y = self.source_examples[index]
        src_x = Image.open(src_x).convert('RGB')
        src_x = self.T(src_x).to(CONFIG.dtype)
        src_y = torch.tensor(src_y).long()

        # randomly sample 'target_examples' to obtain targ_x
        target_idx = torch.randint(len(self.target_examples), size=(1,)).item()
        targ_x, _ = self.target_examples[target_idx]
        targ_x = Image.open(targ_x).convert('RGB')
        targ_x = self.T(targ_x).to(CONFIG.dtype)

        return src_x, src_y, targ_x

######################################################

# [OPTIONAL] TODO: modify 'BaseDataset' for the Domain Generalization setting.
# Hint: combine the examples from the 3 source domains into a single 'examples' list
#class DomainGeneralizationDataset(Dataset):
#    def __init__(self, examples, transform):
#        self.examples = examples
#        self.T = transform
#
#    def __len__(self):
#        return len(self.examples)
#
#    def __getitem__(self, index):
#        x1, x2, x3 = self.examples[index]
#        x1, x2, x3 = self.T(x1), self.T(x2), self.T(x3)
#        targ_x = self.T(targ_x)
#        return x1, x2, x3

######################################################

class SeededDataLoader(DataLoader):
    def __init__(self, dataset: Dataset, batch_size=1, shuffle=None,
                 sampler=None,
                 batch_sampler=None,
                 num_workers=0, collate_fn=None,
                 pin_memory=False, drop_last=False, timeout=0,
                 worker_init_fn=None, multiprocessing_context=None,
                 generator=None, *, prefetch_factor=None, persistent_workers=False,
                 pin_memory_device=""):

        if not CONFIG.use_nondeterministic:
            def seed_worker(worker_id):
                worker_seed = torch.initial_seed() % 2**32
                np.random.seed(worker_seed)
                random.seed(worker_seed)

            generator = torch.Generator()
            generator.manual_seed(CONFIG.seed)

            worker_init_fn = seed_worker

        super().__init__(dataset, batch_size, shuffle, sampler, batch_sampler, num_workers, collate_fn,
                         pin_memory, drop_last, timeout, worker_init_fn, multiprocessing_context, generator,
                         prefetch_factor=prefetch_factor, persistent_workers=persistent_workers,
                         pin_memory_device=pin_memory_device)

#### PACS

In [7]:
def get_transform(size, mean, std, preprocess):
    transform = []
    if preprocess:
        transform.append(T.Resize(256))
        transform.append(T.RandomResizedCrop(size=size, scale=(0.7, 1.0)))
        transform.append(T.RandomHorizontalFlip())
    else:
        transform.append(T.Resize(size))
    transform.append(T.ToTensor())
    transform.append(T.Normalize(mean, std))
    return T.Compose(transform)


def load_data():
    CONFIG.num_classes = 7
    CONFIG.data_input_size = (3, 224, 224)

    # Create transforms
    mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225) # ImageNet Pretrain statistics
    train_transform = get_transform(size=224, mean=mean, std=std, preprocess=True)
    test_transform = get_transform(size=224, mean=mean, std=std, preprocess=False)

    # Load examples
    source_examples, target_examples = [], []
    if CONFIG.experiment in ['baseline', 'random', 'domain_adaptation']:
        # Load source
        f = open(os.path.join(CONFIG.dataset_args['text_root'], f"{CONFIG.dataset_args['source_domain']}.txt"), 'r')
        for line in f:
            line = line.strip().split()
            path, label = line[0].split('/')[0:], int(line[1])
            source_examples.append((os.path.join(CONFIG.dataset_args['images_root'], *path), label))
        f.close()

        # Load target
        f = open(os.path.join(CONFIG.dataset_args['text_root'], f"{CONFIG.dataset_args['target_domain']}.txt"), 'r')
        for line in f:
            line = line.strip().split()
            path, label = line[0].split('/')[0:], int(line[1])
            target_examples.append((os.path.join(CONFIG.dataset_args['images_root'], *path), label))
        f.close()

    # create Dataset object for the baseline and the Random Maps Ablation experiments
    if CONFIG.experiment in ['baseline', 'random']:
        train_dataset = BaseDataset(source_examples, transform=train_transform)
        test_dataset = BaseDataset(target_examples, transform=test_transform)

    # create Dataset object for the Domain Adaptation experiments
    if CONFIG.experiment in ['domain_adaptation']:
        train_dataset = DomainAdaptationDataset(source_examples, target_examples, transform=train_transform)
        test_dataset = BaseDataset(target_examples, transform=test_transform)

    ######################################################

    # Dataloaders
    train_loader = SeededDataLoader(
        train_dataset,
        batch_size=CONFIG.batch_size,
        shuffle=True,
        num_workers=CONFIG.num_workers,
        pin_memory=True,
        persistent_workers=True
    )

    test_loader = SeededDataLoader(
        test_dataset,
        batch_size=CONFIG.batch_size,
        shuffle=False,
        num_workers=CONFIG.num_workers,
        pin_memory=True,
        persistent_workers=True
    )

    return {'train': train_loader, 'test': test_loader}

### Models

#### Base ResNet 18

In [8]:
class BaseResNet18(nn.Module):
    def __init__(self):
        super(BaseResNet18, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)


    def forward(self, x):
        return self.resnet(x)

#### ASH ResNet 18

In [9]:
def get_activation_shaping_hook (mask):

    # Activation Shaping Module as a function that shall be hooked via 'register_forward_hook'
    # The hook captures mask variable from the parent scope, to update it,
    # remove() the hook and register a new one with the updated mask.

    def activation_shaping_hook(module, input, output):

        # print('ASH module registered on: ', module, 'mask shape: ', mask.shape, 'mask sum(): ', mask.sum())

        # binarize both activation map and mask using zero as threshold
        A_binary = torch.where(output<=0, torch.tensor(0.0), torch.tensor(1.0))
        M_binary = torch.where(mask<=0, torch.tensor(0.0), torch.tensor(1.0))

        # return the element-wise product of activation map and mask
        shaped_output = A_binary * M_binary
        return shaped_output

    return activation_shaping_hook

######################################################

# 'BaseResNet18' including the Activation Shaping Module
class ASHResNet18(nn.Module):
    def __init__(self):
        super(ASHResNet18, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)

        self.hook_handles = {}


    def register_activation_shaping_hook(self, layer_name, mask):
        hook = get_activation_shaping_hook(mask)

        for name, module in self.resnet.named_modules():
            if ((isinstance(module, nn.ReLU) or isinstance(module, nn.Conv2d) or isinstance(module, nn.BatchNorm2d)) and name == layer_name):
                if not name in self.hook_handles:
                    print('Insert activation shaping layer after ', name, module)
                    self.hook_handles[name] = module.register_forward_hook(hook)


    def remove_activation_shaping_hook(self, name):
        if name in self.hook_handles and self.hook_handles[name] is not None:
            handle = self.hook_handles[name]
            handle.remove()


    def forward(self, x):
        return self.resnet(x)

#### ASH ResNet 18 for Domain Adaptation

In [10]:
# 'BaseResNet18' including the Activation Shaping Module for Domain Adaptation
class ASHResNet18DomainAdaptation(nn.Module):
    def __init__(self, module_placement=None):
        super(ASHResNet18DomainAdaptation, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)

        self.module_placement = module_placement
        if module_placement==[]: self.module_placement = None
        self.activation_maps = {}
        self.activation_map_hook_handles = {}


    def get_extract_activation_map_hook1 (self, layer_name):
        def extract_activation_map_hook(module, input, output):
            self.activation_maps[layer_name] = output.clone().detach()
        return extract_activation_map_hook


    def get_extract_activation_map_hook2 (self, layer_name):
        def extract_activation_map_hook(module, input, output):
            A = torch.where(output<=0, torch.tensor(0.0), torch.tensor(1.0))
            M = self.activation_maps[layer_name]
            A = torch.where(M<=0, torch.tensor(0.0), torch.tensor(1.0))
            shaped_output = A * M
            return shaped_output
        return extract_activation_map_hook


    def forward(self, x, targ_x=None):
        if targ_x is not None: # First step for point 3
            if self.module_placement is not None:
                # Register a hook and perform a forward pass to store the target domain activation maps
                for layer_name, module in self.resnet.named_modules():
                    if (isinstance(module, nn.Conv2d) and layer_name in self.module_placement):
                        #print('Register a hook to store activation map of layer ', layer_name, module)
                        hook = self.get_extract_activation_map_hook1(layer_name)
                        self.activation_map_hook_handles[layer_name] = module.register_forward_hook(hook)
                _ = self.resnet(targ_x)
        else: # Second step for point 3
            if self.module_placement is not None:
                # Register a hook and perform a forward pass to compute source labels
                for layer_name, module in self.resnet.named_modules():
                    if (isinstance(module, nn.Conv2d) and layer_name in self.module_placement):
                        #print('Register a hook to store activation map of layer ', layer_name, module)
                        hook = self.get_extract_activation_map_hook2(layer_name)
                        self.activation_map_hook_handles[layer_name] = module.register_forward_hook(hook)
        output = self.resnet(x)

        return output

### Run

#### Evaluate

In [11]:
@torch.no_grad()
def evaluate(model, data):
    model.eval()

    acc_meter = Accuracy(task='multiclass', num_classes=CONFIG.num_classes)
    acc_meter = acc_meter.to(CONFIG.device)

    loss = [0.0, 0]
    for x, y in tqdm(data):
        with torch.autocast(device_type=CONFIG.device, dtype=torch.float16, enabled=True):
            x, y = x.to(CONFIG.device), y.to(CONFIG.device)
            logits = model(x)
            acc_meter.update(logits, y)
            loss[0] += F.cross_entropy(logits, y).item()
            loss[1] += x.size(0)

    accuracy = acc_meter.compute()
    loss = loss[0] / loss[1]
    logging.info(f'Accuracy: {100 * accuracy:.2f} - Loss: {loss}')
    print(f'Accuracy: {100 * accuracy:.2f} - Loss: {loss} \n')

#### Train

In [12]:
def train(model, data):
    # Create optimizers & schedulers
    optimizer = torch.optim.SGD(model.parameters(), weight_decay=0.0005, momentum=0.9, nesterov=True, lr=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=int(CONFIG.epochs * 0.8), gamma=0.1)
    scaler = torch.cuda.amp.GradScaler(enabled=True)

    # Load checkpoint (if it exists)
    cur_epoch = 0
    if os.path.exists(os.path.join('record', CONFIG.experiment_name, 'last.pth')):
        checkpoint = torch.load(os.path.join('record', CONFIG.experiment_name, 'last.pth'))
        cur_epoch = checkpoint['epoch']
        optimizer.load_state_dict(checkpoint['optimizer'])
        scheduler.load_state_dict(checkpoint['scheduler'])
        model.load_state_dict(checkpoint['model'])

    # Optimization loop
    for epoch in range(cur_epoch, CONFIG.epochs):
        model.train()

        for batch_idx, batch in enumerate(tqdm(data['train'])):

            # Compute loss
            with torch.autocast(device_type=CONFIG.device, dtype=torch.float16, enabled=True):

                if CONFIG.experiment in ['baseline']:
                    x, y = batch
                    x, y = x.to(CONFIG.device), y.to(CONFIG.device)
                    loss = F.cross_entropy(model(x), y)

                ######################################################

                elif CONFIG.experiment in ['random']:
                    x, y = batch
                    x, y = x.to(CONFIG.device), y.to(CONFIG.device)
                    loss = F.cross_entropy(model(x), y)

                elif CONFIG.experiment in ['domain_adaptation']:
                    src_x, src_y, targ_x = batch
                    x, y = src_x.to(CONFIG.device), src_y.to(CONFIG.device)
                    targ_x = targ_x.to(CONFIG.device)
                    loss = F.cross_entropy(model(x, targ_x=targ_x), y)

                ######################################################

            # Optimization step
            scaler.scale(loss / CONFIG.grad_accum_steps).backward()

            if ((batch_idx + 1) % CONFIG.grad_accum_steps == 0) or (batch_idx + 1 == len(data['train'])):
                scaler.step(optimizer)
                optimizer.zero_grad(set_to_none=True)
                scaler.update()

        scheduler.step()

        # Test current epoch
        logging.info(f'[TEST @ Epoch={epoch}]')
        evaluate(model, data['test'])

        # Save checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict(),
            'model': model.state_dict()
        }
        torch.save(checkpoint, os.path.join('record', CONFIG.experiment_name, 'last.pth'))

#### Main

In [13]:
layer_output_shapes = {
    'layer1.0.conv1': (64, 56, 56),         'layer1.0.relu': (64, 56, 56),          'layer1.0.conv2': (64, 56, 56),
    'layer1.1.conv1': (64, 56, 56),         'layer1.1.bn1': (64, 56, 56),           'layer1.1.relu': (64, 56, 56),          'layer1.1.conv2': (64, 56, 56),         'layer1.1.bn2': (64, 56, 56),
    'layer2.0.conv1': (128, 28, 28),        'layer2.0.relu': (128, 28, 28),         'layer2.0.conv2': (128, 28, 28),
    'layer2.1.conv1': (128, 28, 28),        'layer2.1.relu': (128, 28, 28),         'layer2.1.conv2': (128, 28, 28),
    'layer3.0.conv1': (256, 14, 14),        'layer3.0.relu': (256, 14, 14),         'layer3.0.conv2': (256, 14, 14),
    'layer3.1.conv1': (256, 14, 14),        'layer3.1.relu': (256, 14, 14),         'layer3.1.conv2': (256, 14, 14),
    'layer4.0.conv1': (512, 7, 7),          'layer4.0.relu': (512, 7, 7),           'layer4.0.conv2': (512, 7, 7),
    'layer4.1.conv1': (512, 7, 7),          'layer4.1.relu': (512, 7, 7),           'layer4.1.conv2': (512, 7, 7)
}


def create_random_mask(layer_name = 'layer4.1.conv2', mask_out_ratio = 0.0):
    # create a mask tensor with a given ratio of zeros
    print('mask_out_ratio: ',mask_out_ratio)
    layer_output_shape = layer_output_shapes[layer_name]

    rand_mat = torch.rand(layer_output_shape).to(CONFIG.device)
    mask = torch.where(rand_mat <= mask_out_ratio, 0.0, 1.0).to(CONFIG.device)
    return mask


def main ():
    # Load dataset
    data = load_data()

    # Load model
    if CONFIG.experiment in ['baseline']:
        model = BaseResNet18()

    ######################################################

    elif CONFIG.experiment in ['random']:
        model = ASHResNet18()
        module_placement = CONFIG.experiment_args['module_placement']
        mask_out_ratio = CONFIG.experiment_args['mask_out_ratio']

        for layer_name in module_placement:
            random_mask = create_random_mask(layer_name = layer_name, mask_out_ratio = mask_out_ratio)
            print('layer_name', layer_name, 'random_mask sum(): ', random_mask.sum())
            model.register_activation_shaping_hook(layer_name, random_mask)

    elif CONFIG.experiment in ['domain_adaptation']:
        module_placement = CONFIG.experiment_args['module_placement']
        model = ASHResNet18DomainAdaptation(module_placement)

    ######################################################

    model.to(CONFIG.device)

    if not CONFIG.test_only:
        train(model, data)
    else:
        evaluate(model, data['test'])


## 0 - Reproduce the Baseline

### 0.1 - Baseline

#### 0.1.1 - Art painting &rightarrow; Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'baseline',
    'experiment_name': 'baseline/cartoon/',
    'experiment_args': '{}',
    'dataset_args': { 'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='baseline', experiment_name='baseline/cartoon/', experiment_args='{}', dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 151MB/s]
100%|██████████| 19/19 [00:10<00:00,  1.87it/s]


Accuracy: 30.76 - Loss: 0.014742844086458252 



100%|██████████| 19/19 [00:07<00:00,  2.57it/s]


Accuracy: 41.89 - Loss: 0.013021460470486013 



...

Accuracy: 54.22 - Loss: 0.010744376047657414 



100%|██████████| 19/19 [00:07<00:00,  2.61it/s]


Accuracy: 54.44 - Loss: 0.010650597334721797 



100%|██████████| 19/19 [00:07<00:00,  2.64it/s]


Accuracy: 54.65 - Loss: 0.010656903227695833 



#### 0.1.2 - Art painting &rightarrow; Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'baseline',
    'experiment_name': 'baseline/sketch/',
    'experiment_args': '{}',
    'dataset_args': { 'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='baseline', experiment_name='baseline/sketch/', experiment_args='{}', dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)


100%|██████████| 31/31 [00:12<00:00,  2.39it/s]


Accuracy: 10.61 - Loss: 0.016416618768784738 



100%|██████████| 31/31 [00:11<00:00,  2.75it/s]


Accuracy: 24.10 - Loss: 0.015480421735725199 



...

Accuracy: 39.76 - Loss: 0.015285405221559462 



100%|██████████| 31/31 [00:10<00:00,  2.83it/s]


Accuracy: 40.09 - Loss: 0.015074179809067383 



100%|██████████| 31/31 [00:11<00:00,  2.60it/s]


Accuracy: 40.34 - Loss: 0.015126618144469093 



#### 0.1.3 - Art painting &rightarrow; Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'baseline',
    'experiment_name': 'baseline/photo/',
    'experiment_args': '{}',
    'dataset_args': { 'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='baseline', experiment_name='baseline/photo/', experiment_args='{}', dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)


100%|██████████| 14/14 [00:05<00:00,  2.60it/s]


Accuracy: 60.84 - Loss: 0.011621268852028304 



100%|██████████| 14/14 [00:05<00:00,  2.52it/s]


Accuracy: 79.70 - Loss: 0.006602486129292471 



...

Accuracy: 95.81 - Loss: 0.0010731483313316356 



100%|██████████| 14/14 [00:05<00:00,  2.67it/s]


Accuracy: 95.81 - Loss: 0.0010843102440996441 



100%|██████████| 14/14 [00:05<00:00,  2.41it/s]


Accuracy: 95.87 - Loss: 0.001070433580679094 



## 1 - Activation Shaping module

### 1.1 - ASH module after last convolutional layer

#### 1.1.1 - Art painting &rightarrow; Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/after_last_conv/cartoon/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/after_last_conv/cartoon/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
layer_name layer4.1.conv2 random_mask sum():  tensor(22611., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 19/19 [00:06<00:00,  2.76it/s]


Accuracy: 20.05 - Loss: 0.01588940671279569 



100%|██████████| 19/19 [00:06<00:00,  2.92it/s]


Accuracy: 27.86 - Loss: 0.014836063462312718 



...

Accuracy: 43.52 - Loss: 0.011583674063353002 



100%|██████████| 19/19 [00:07<00:00,  2.50it/s]


Accuracy: 43.30 - Loss: 0.011567245927280126 



100%|██████████| 19/19 [00:06<00:00,  2.98it/s]


Accuracy: 43.26 - Loss: 0.011542623907124223 



#### 1.1.2 - Art painting &rightarrow; Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/after_last_conv/sketch/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/after_last_conv/sketch/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
layer_name layer4.1.conv2 random_mask sum():  tensor(22611., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 31/31 [00:12<00:00,  2.45it/s]


Accuracy: 6.41 - Loss: 0.017048950746302752 



100%|██████████| 31/31 [00:12<00:00,  2.58it/s]


Accuracy: 10.38 - Loss: 0.01605362366040143 



...

Accuracy: 38.23 - Loss: 0.014106580943483589 



100%|██████████| 31/31 [00:11<00:00,  2.82it/s]


Accuracy: 38.25 - Loss: 0.01411462133547889 



100%|██████████| 31/31 [00:11<00:00,  2.62it/s]


Accuracy: 38.28 - Loss: 0.014157276597754832 



#### 1.1.3 - Art painting &rightarrow; Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/after_last_conv/photo/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/after_last_conv/photo/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
layer_name layer4.1.conv2 random_mask sum():  tensor(22611., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 14/14 [00:05<00:00,  2.70it/s]


Accuracy: 33.35 - Loss: 0.014960597018281856 



100%|██████████| 14/14 [00:05<00:00,  2.36it/s]


Accuracy: 47.66 - Loss: 0.012346574146590548 



...

Accuracy: 91.38 - Loss: 0.003084783144518287 



100%|██████████| 14/14 [00:04<00:00,  3.11it/s]


Accuracy: 91.38 - Loss: 0.0030988118650313624 



100%|██████████| 14/14 [00:04<00:00,  3.10it/s]


Accuracy: 91.50 - Loss: 0.0030856483219983337 



### 1.2 - ASH module after each convolutional layer

#### 1.2.1 - Art Painting &rightarrow; Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/after_each_conv/cartoon/',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/after_each_conv/cartoon/', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
layer_name layer1.0.conv1 random_mask sum():  tensor(180587., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.1
layer_name layer1.0.conv2 random_mask sum():  tensor(180685., device='cuda:0')
Insert

100%|██████████| 19/19 [00:08<00:00,  2.22it/s]


Accuracy: 15.10 - Loss: 0.015846378719847357 



100%|██████████| 19/19 [00:06<00:00,  2.90it/s]


Accuracy: 16.25 - Loss: 0.01595945066356008 



...

Accuracy: 22.48 - Loss: 0.015219537064483955 



100%|██████████| 19/19 [00:06<00:00,  2.89it/s]


Accuracy: 22.10 - Loss: 0.015301868777226262 



100%|██████████| 19/19 [00:07<00:00,  2.47it/s]


Accuracy: 22.06 - Loss: 0.015217787576616827 



### 1.3 - ASH module after every three convolutional layers


#### 1.3.1 - Art Painting &rightarrow; Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/every_three_conv/cartoon/',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/every_three_conv/cartoon/', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
layer_name layer1.0.conv1 random_mask sum():  tensor(180587., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.1
layer_name layer1.1.conv2 random_mask sum():  tensor(180685., device='cuda:0')
Insert activation shaping layer after  layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.1
layer_name layer2.1.conv1 random

100%|██████████| 19/19 [00:07<00:00,  2.59it/s]


Accuracy: 11.69 - Loss: 0.016672733904151787 



100%|██████████| 19/19 [00:06<00:00,  2.92it/s]


Accuracy: 21.89 - Loss: 0.015613385994279751 



...

Accuracy: 31.61 - Loss: 0.015218264820632673 



100%|██████████| 19/19 [00:06<00:00,  2.75it/s]


Accuracy: 31.78 - Loss: 0.01524022328365378 



100%|██████████| 19/19 [00:06<00:00,  2.99it/s]


Accuracy: 31.14 - Loss: 0.01518758078687427 



## 2 - Random Activation Maps

Before running these cells: delete the corresponding last.pth file if present.

### 2.1 - mask out ratio = 0.4, module placement: layer4.1.conv2

#### 2.1.1 - Art Painting &rightarrow; Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case1/cartoon/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case1/cartoon/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.4
layer_name layer4.1.conv2 random_mask sum():  tensor(15133., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 19/19 [00:08<00:00,  2.37it/s]


Accuracy: 20.52 - Loss: 0.015954948224305293 



100%|██████████| 19/19 [00:06<00:00,  2.83it/s]


Accuracy: 27.99 - Loss: 0.014950866178440967 



...

Accuracy: 43.81 - Loss: 0.011543101875855248 



100%|██████████| 19/19 [00:07<00:00,  2.60it/s]


Accuracy: 43.81 - Loss: 0.01152608472434327 



100%|██████████| 19/19 [00:07<00:00,  2.62it/s]


Accuracy: 44.07 - Loss: 0.011509414317257983 



#### 2.1.2 - Art Painting &rightarrow; Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case1/sketch/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case1/sketch/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.4
layer_name layer4.1.conv2 random_mask sum():  tensor(15133., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 31/31 [00:12<00:00,  2.42it/s]


Accuracy: 4.53 - Loss: 0.017142348418135046 



100%|██████████| 31/31 [00:11<00:00,  2.67it/s]


Accuracy: 8.63 - Loss: 0.01608260162127294 



...

Accuracy: 39.58 - Loss: 0.013864052329059774 



100%|██████████| 31/31 [00:11<00:00,  2.62it/s]


Accuracy: 39.73 - Loss: 0.013884821748818475 



100%|██████████| 31/31 [00:11<00:00,  2.62it/s]


Accuracy: 39.76 - Loss: 0.013908793465664563 



#### 2.1.3 - Art Painting &rightarrow; Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case1/photo/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case1/photo/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.4
layer_name layer4.1.conv2 random_mask sum():  tensor(15133., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 14/14 [00:05<00:00,  2.47it/s]


Accuracy: 33.29 - Loss: 0.0152633750510073 



100%|██████████| 14/14 [00:04<00:00,  3.04it/s]


Accuracy: 46.59 - Loss: 0.013171693593442084 



...

Accuracy: 90.78 - Loss: 0.003655197993367018 



100%|██████████| 14/14 [00:05<00:00,  2.46it/s]


Accuracy: 90.84 - Loss: 0.0036668386377260356 



100%|██████████| 14/14 [00:05<00:00,  2.75it/s]


Accuracy: 91.02 - Loss: 0.0036496157596211235 



### 2.2 - mask out ratio = 0.55, module placement: layer4.1.conv2

#### 2.2.1 - Art Painting &rightarrow; Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case2/cartoon/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case2/cartoon/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.55
layer_name layer4.1.conv2 random_mask sum():  tensor(11245., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 19/19 [00:06<00:00,  2.74it/s]


Accuracy: 19.88 - Loss: 0.01594231249732776 



100%|██████████| 19/19 [00:07<00:00,  2.53it/s]


Accuracy: 27.69 - Loss: 0.015005142310373613 



...

Accuracy: 43.98 - Loss: 0.011562336528667411 



100%|██████████| 19/19 [00:06<00:00,  3.11it/s]


Accuracy: 44.03 - Loss: 0.011531739231872885 



100%|██████████| 19/19 [00:06<00:00,  2.96it/s]


Accuracy: 44.03 - Loss: 0.011525974821922315 



#### 2.2.2 - Art Painting &rightarrow; Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case2/sketch/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case2/sketch/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.55
layer_name layer4.1.conv2 random_mask sum():  tensor(11245., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 31/31 [00:12<00:00,  2.41it/s]


Accuracy: 4.38 - Loss: 0.01711246183914336 



100%|██████████| 31/31 [00:11<00:00,  2.60it/s]


Accuracy: 8.25 - Loss: 0.016080539776915415 



...

Accuracy: 39.09 - Loss: 0.013798011232193083 



100%|██████████| 31/31 [00:11<00:00,  2.61it/s]


Accuracy: 39.07 - Loss: 0.01380500322566684 



100%|██████████| 31/31 [00:11<00:00,  2.62it/s]


Accuracy: 39.25 - Loss: 0.013818536832697308 



#### 2.2.3 - Art Painting &rightarrow; Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case2/photo/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case2/photo/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.55
layer_name layer4.1.conv2 random_mask sum():  tensor(11245., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 14/14 [00:06<00:00,  2.16it/s]


Accuracy: 32.34 - Loss: 0.015374206854197794 



100%|██████████| 14/14 [00:05<00:00,  2.42it/s]


Accuracy: 45.57 - Loss: 0.01374882302598325 



...

Accuracy: 90.18 - Loss: 0.004072063893615129 



100%|██████████| 14/14 [00:05<00:00,  2.62it/s]


Accuracy: 90.12 - Loss: 0.004075257881672796 



100%|██████████| 14/14 [00:04<00:00,  3.03it/s]


Accuracy: 90.00 - Loss: 0.004075009576574771 



### 2.3 - mask out ratio = 0.75, module placement: layer4.1.conv2

#### 2.3.1 - Art Painting &rightarrow; Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case3/cartoon/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case3/cartoon/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.75
layer_name layer4.1.conv2 random_mask sum():  tensor(6311., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 19/19 [00:06<00:00,  2.72it/s]


Accuracy: 18.77 - Loss: 0.015922175993691532 



100%|██████████| 19/19 [00:07<00:00,  2.54it/s]


Accuracy: 23.85 - Loss: 0.015261314529607727 



...

Accuracy: 43.13 - Loss: 0.012103678525104458 



100%|██████████| 19/19 [00:06<00:00,  3.00it/s]


Accuracy: 43.09 - Loss: 0.01206385011152196 



100%|██████████| 19/19 [00:07<00:00,  2.57it/s]


Accuracy: 43.05 - Loss: 0.012056090268258756 



#### 2.3.2 - Art Painting &rightarrow; Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case3/sketch/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case3/sketch/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.75
layer_name layer4.1.conv2 random_mask sum():  tensor(6311., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 31/31 [00:12<00:00,  2.42it/s]


Accuracy: 4.22 - Loss: 0.016792870927142198 



100%|██████████| 31/31 [00:11<00:00,  2.64it/s]


Accuracy: 6.26 - Loss: 0.01616434141951804 



...

Accuracy: 36.85 - Loss: 0.014143930011864805 



100%|██████████| 31/31 [00:11<00:00,  2.59it/s]


Accuracy: 37.24 - Loss: 0.014145380184225042 



100%|██████████| 31/31 [00:11<00:00,  2.60it/s]


Accuracy: 37.31 - Loss: 0.014148369488384804 



#### 2.3.3 - Art Painting &rightarrow; Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case3/photo/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case3/photo/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.75
layer_name layer4.1.conv2 random_mask sum():  tensor(6311., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 14/14 [00:06<00:00,  2.31it/s]


Accuracy: 25.75 - Loss: 0.01563600858528457 



100%|██████████| 14/14 [00:04<00:00,  2.91it/s]


Accuracy: 38.38 - Loss: 0.014700142994612277 



...

Accuracy: 88.14 - Loss: 0.0052436300023587165 



100%|██████████| 14/14 [00:05<00:00,  2.42it/s]


Accuracy: 88.02 - Loss: 0.005245980995143959 



100%|██████████| 14/14 [00:05<00:00,  2.64it/s]


Accuracy: 88.02 - Loss: 0.005236013992103988 



## 3 - Adapting Activation Maps across Domains

Before running these cells: delete the corresponding last.pth file if present.

### 3.1 - module placement: layer2.1.conv2

#### 3.1.1 - Art Painting &rightarrow; Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'domain_adaptation',
    'experiment_name': 'domain_adaptation/cartoon/',
    'experiment_args': {'module_placement': ['layer2.1.conv2']},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='domain_adaptation', experiment_name='domain_adaptation/cartoon/', experiment_args={'module_placement': ['layer2.1.conv2']}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)


100%|██████████| 19/19 [00:08<00:00,  2.32it/s]


Accuracy: 18.22 - Loss: 0.015873363807339717 



100%|██████████| 19/19 [00:07<00:00,  2.59it/s]


Accuracy: 49.36 - Loss: 0.011627306252616258 



...

Accuracy: 68.05 - Loss: 0.007260306347352246 



100%|██████████| 19/19 [00:32<00:00,  1.72s/it]


Accuracy: 68.09 - Loss: 0.007216504511165944 



100%|██████████| 19/19 [00:33<00:00,  1.76s/it]


Accuracy: 68.77 - Loss: 0.00713463635549407 



#### 3.1.2 - Art Painting &rightarrow; Sketch

In [14]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'domain_adaptation',
    'experiment_name': 'domain_adaptation/sketch/',
    'experiment_args': {'module_placement': ['layer2.1.conv2']},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='domain_adaptation', experiment_name='domain_adaptation/sketch/', experiment_args={'module_placement': ['layer2.1.conv2']}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 99.1MB/s]
100%|██████████| 31/31 [00:13<00:00,  2.37it/s]


Accuracy: 17.21 - Loss: 0.016646513656126873 



100%|██████████| 31/31 [00:12<00:00,  2.45it/s]


Accuracy: 19.73 - Loss: 0.014952954641581492 



...

Accuracy: 51.13 - Loss: 0.010565826966581104 



100%|██████████| 31/31 [01:16<00:00,  2.47s/it]


Accuracy: 51.41 - Loss: 0.010462023558644552 



100%|██████████| 31/31 [01:20<00:00,  2.59s/it]


Accuracy: 51.23 - Loss: 0.010565697206253008 



#### 3.1.3 - Art Painting &rightarrow; Photo

In [14]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'domain_adaptation',
    'experiment_name': 'domain_adaptation/photo/',
    'experiment_args': {'module_placement': ['layer2.1.conv2']},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='domain_adaptation', experiment_name='domain_adaptation/photo/', experiment_args={'module_placement': ['layer2.1.conv2']}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 78.9MB/s]
100%|██████████| 14/14 [00:05<00:00,  2.72it/s]


Accuracy: 21.56 - Loss: 0.01595942938399172 



100%|██████████| 14/14 [00:05<00:00,  2.79it/s]


Accuracy: 80.84 - Loss: 0.006798694376460092 



...

Accuracy: 95.69 - Loss: 0.0010147935989686472 



100%|██████████| 14/14 [00:19<00:00,  1.37s/it]


Accuracy: 95.87 - Loss: 0.0010484497838034602 



100%|██████████| 14/14 [00:19<00:00,  1.42s/it]


Accuracy: 95.81 - Loss: 0.001045124056542705 



## Ext. 2 - Binarization Ablation